Let us first load the necessary packages for this introductory sheet of code.

In [ ]:
import numpy as np
from pulser import Pulse, Sequence, Register

# 1 - Creating the register

The Register defines the positions of the atoms and their names of each one. There are multiple ways of defining a Register, the most customizable one being to create a dictionary that associates a name (the key) to a coordinate (the value).

In [ ]:
L = 4
square = np.array([[i, j] for i in range(L) for j in range(L)], dtype=float)

In [ ]:
square -= np.mean(square, axis=0)
square *= 5

In [ ]:
qubits = dict(enumerate(square))

In [ ]:
qubits

The Register class provides some useful features, like the ability to visualise the array.

In [ ]:
reg = Register(qubits)
reg.draw()

If one doesn’t particularly care about the name given to the qubits, one can also create a Register just from a list of coordinates. In this case, the qubit ID’s are just numbered, starting from 0, in the order they are provided in, with the option of adding a common prefix before each number. Also, it automatically centers the entire array around the origin, an option that can be disabled if desired.

In [ ]:
# Alternative way of doing
reg2 = Register.from_coordinates(square, prefix='q')
reg2.draw()

Furthermore, there are also built-in class methods from creation of common array patterns, namely: 
- Square lattices in rectangular or square shapes
- Triangular lattices

We could, thus, create the same square array as before by doing:

In [ ]:
# Or with built in methods:
reg3 = Register.square(4, spacing = 5)
reg3.draw()

# 2 - Initializing the Sequence

To create a Sequence, one has to provide it with the Register instance and the device in which the sequence will be executed. The chosen device will dictate whether the register is valid or not.

We import the device (in this case, DigitalAnalogDevice) from pulser.devices and initialize our sequence with the freshly created register:

In [ ]:
from pulser.devices import AnalogDevice, DigitalAnalogDevice

For current generation experimentations, AnalogDevice or MockDevice is recommended. Otherwise, DigitalAnalogDevice can also be used with local detuning modulation.

# 3 - Declaring the channels that will be used

Inspecting what channels are available on this device:

In [ ]:
seq = Sequence(reg, AnalogDevice)

In [ ]:
seq.available_channels

In [ ]:
seq = Sequence(reg, DigitalAnalogDevice)

In [ ]:
seq.available_channels

In [ ]:
# Suppose we want to work only with local channels
seq.declare_channel("ch0", "rydberg_global")
seq.available_channels # Raman_local is no longer available

In [ ]:
seq.declare_channel("ch1", "rydberg_local", initial_target=4)
seq.available_channels

At any time, we can also consult which channels were declared, their specifications and the name they were given by calling:


In [ ]:
seq.declared_channels

# 4 - Composing the Sequence

Every channel needs to start with a target. For Global channels this is predefined to be all qubits in the device, but for Local channels this has to be defined. This initial target can be set through at channel declaration, or it can be done through the standard target instruction.

In [ ]:
# Let's start with a very simple pulse
simple_pulse = Pulse.ConstantPulse(duration = 200, amplitude = 2, detuning = -10, phase = 0)

# Let's add this pulse to "ch0"
seq.add(simple_pulse, "ch0")

In [ ]:
# Suppose we want to freeze "ch1" for 100 ns while "ch0" is doing its pulse
seq.delay(100, "ch1")

In [ ]:
print(seq)

We can also draw the sequence, for a more visual representation:


In [ ]:
seq.draw()

In [ ]:
# Now suppose we want to add to "ch1" a more complex pulse with 
# omega != cst and delta != cst
from pulser.waveforms import BlackmanWaveform, RampWaveform
amp_wf = BlackmanWaveform(duration=1000, area=np.pi/2)
detuning_wf = RampWaveform(duration=1000, start=-20, stop=20)

In [ ]:
amp_wf.integral

We can visualize a waveform by calling:


In [ ]:
amp_wf.draw()

In [ ]:
detuning_wf.draw()

In [ ]:
complex_pulse = Pulse(amplitude = amp_wf, detuning = detuning_wf, phase = 0)

In [ ]:
complex_pulse.draw()

In [ ]:
seq.add(complex_pulse, "ch1")

In [ ]:
print(seq)

In [ ]:
seq.draw()

Now, let’s see how the Sequence builder handles conflicts (i.e. two channels acting on the same qubit at once). 

In [ ]:
# Situation with two pulses from two different channels  
# Default protocol is 'min-delay'
seq.add(complex_pulse, 'ch0')
print(seq)
seq.draw()

In [ ]:
# Let's see now another protocol named 'wait-for-all'
seq.target(0, 'ch1')

In [ ]:
seq.add(simple_pulse, 'ch1', protocol='min-delay')
seq.add(simple_pulse, 'ch1', protocol='wait-for-all')
print(seq)
seq.draw()


In [ ]:
# Another protocol is the 'no-delay' (not recommended)
seq.add(complex_pulse, "ch0", protocol='no-delay')
print(seq)
seq.draw()

# 5 - Measurement

To finish a sequence, we measure it. A measurement signals the end of a sequence, so after it no more changes are possible. When measuring, one has to select the desired measurement basis. The availabe options depend on the device and can be consulted by calling:


In [ ]:
# Measurement = final element of a sequence
# All the qubits are measured
DigitalAnalogDevice.supported_bases

In [ ]:
seq.measure(basis='ground-rydberg')
seq.draw()